In [6]:
import os
from collections import OrderedDict
import nltk
nltk.download('punkt')
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fashn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
corpus_path = 'Corpus 2'
stopwords_file = 'Stopwords.txt'

In [8]:

files=os.path.join(corpus_path)

# getting list of all files
corpus_files = os.listdir(corpus_path)
print(corpus_files)

documents=[]
queries_files = []

for file in corpus_files:
    if 'Document' in file:
        documents.append(file)
    elif 'Query' in file:
        queries_files.append(file)
        

print(documents)
print(queries_files)


['Document 1.txt', 'Document 10.txt', 'Document 2.txt', 'Document 3.txt', 'Document 4.txt', 'Document 5.txt', 'Document 6.txt', 'Document 7.txt', 'Document 8.txt', 'Document 9.txt', 'Query 1.txt', 'Query 2.txt', 'Query 3.txt', 'Query 4.txt']
['Document 1.txt', 'Document 10.txt', 'Document 2.txt', 'Document 3.txt', 'Document 4.txt', 'Document 5.txt', 'Document 6.txt', 'Document 7.txt', 'Document 8.txt', 'Document 9.txt']
['Query 1.txt', 'Query 2.txt', 'Query 3.txt', 'Query 4.txt']


In [9]:
# Load stopwords
with open(stopwords_file, 'r') as f:
    stopwords = set(f.read().splitlines())

print(stopwords)

{'up', 'him', 'did', 'were', 'ours', 'after', 'have', 'from', 'when', 't', 'them', 'theirs', 'then', 'you', 'each', 'themselves', 'other', 'yourself', 'at', 'over', 'with', 'during', 'while', 'not', 'these', 'but', 'or', 'our', 'who', 'has', 'be', 'a', 'further', 'ourselves', 'here', 'under', 'myself', 'off', 'being', 'no', 'don', 'very', 'its', 'few', 'if', 'can', 'that', 'in', 'been', 'their', 'once', 'most', 'itself', 'she', 'was', 'such', 'will', 'again', 'himself', 'some', 'should', 'had', 'the', 'as', 'about', 'before', 'there', 'by', 'so', 'on', 'those', 'whom', 'than', 'both', 'only', 'and', 'how', 'above', 'for', 'does', 'between', 'hers', 'is', 'against', 'more', 'out', 'all', 'i', 'this', 'your', 'because', 'me', 'yourselves', 'to', 'having', 'an', 'doing', 'nor', 'her', 'until', 'am', 'do', 'it', 'which', 'we', 'into', 'too', 'of', 'any', 'his', 'through', 'below', 'he', 'same', 'they', 'where', 'yours', 'down', 'own', 'why', 'my', 'now', 'what', 's', 'are', 'herself', 'jus

In [22]:
# Tokenize and preprocess queries
queries = {}
terms = set()
for filename in queries_files:
    with open(os.path.join(corpus_path, filename), 'r') as f:
        text = f.read()
        tokens = nltk.word_tokenize(text.lower())
        tokens = [token for token in tokens if token.isalpha() and token not in stopwords]
        porter_stemmer = nltk.PorterStemmer()
        stems = [porter_stemmer.stem(token) for token in tokens]
        queries[filename]=stems+9*-
        for term in stems:
            terms.add(term)

print(queries)
print(terms)

{'Query 1.txt': ['inform', 'retriev', 'activ'], 'Query 2.txt': ['medicin', 'imag', 'retriev'], 'Query 3.txt': ['best', 'field', 'comput'], 'Query 4.txt': ['inform', 'relat', 'mri', 'ct', 'scan']}
{'imag', 'medicin', 'relat', 'activ', 'scan', 'comput', 'best', 'mri', 'retriev', 'inform', 'field', 'ct'}


In [27]:
# initializing dataframe
matrix = pd.DataFrame(terms,columns=['Terms'])
new = ['idf']
temp = 'tf-idf'
for i in range(len(queries)):
    new.append(temp + ' '+str(i+1))

matrix[queries_files] = pd.DataFrame([[None]*len(queries_files)], index=matrix.index)
matrix[new]=pd.DataFrame([[None]*len(new)], index=matrix.index)

matrix

,Terms,Query 1.txt,Query 2.txt,Query 3.txt,Query 4.txt,idf,tf-idf 1,tf-idf 2,tf-idf 3,tf-idf 4
0,imag,None,None,None,None,None,None,None,None,None
1,medicin,None,None,None,None,None,None,None,None,None
2,relat,None,None,None,None,None,None,None,None,None
3,activ,None,None,None,None,None,None,None,None,None
4,scan,None,None,None,None,None,None,None,None,None
5,comput,None,None,None,None,None,None,None,None,None
6,best,None,None,None,None,None,None,None,None,None
7,mri,None,None,None,None,None,None,None,None,None
8,retriev,None,None,None,None,None,None,None,None,None
9,inform,None,None,None,None,None,None,None,None,None


In [31]:
import math

# print(len(documents))
# print(len(corpus[documents[0]]))

for idx, term in matrix['Terms'].iteritems():
    df = 0
    for doc in queries_files:
        if term in queries[doc]:
            df+=1
#             tf calculation
            tf = queries[doc].count(term) / len(queries[doc])
#             matrix.loc[idx, doc] = tf
            if tf>0:
                matrix.loc[idx, doc] =1 + math.log10(tf)
            else:
                matrix.loc[idx, doc] = 0
        else:
            matrix.loc[idx, doc] = 0
    
#     idf
    idf = math.log10(len(queries_files)/df)
    matrix.loc[idx, 'idf'] = idf
    
#     tf-idf
    temp = 'tf-idf'
    i = 0
    for doc in queries_files:
        temp1 = temp + ' '+str(i+1)
        tf_idf = matrix.loc[idx,doc] * idf
        matrix.loc[idx, temp1]=tf_idf
        i+=1
        
matrix

,Terms,Query 1.txt,Query 2.txt,Query 3.txt,Query 4.txt,idf,tf-idf 1,tf-idf 2,tf-idf 3,tf-idf 4
0,imag,0,0.522879,0,0,0.60206,0.0,0.314804,0.0,0.0
1,medicin,0,0.522879,0,0,0.60206,0.0,0.314804,0.0,0.0
2,relat,0,0,0,0.30103,0.60206,0.0,0.0,0.0,0.181238
3,activ,0.522879,0,0,0,0.60206,0.314804,0.0,0.0,0.0
4,scan,0,0,0,0.30103,0.60206,0.0,0.0,0.0,0.181238
5,comput,0,0,0.522879,0,0.60206,0.0,0.0,0.314804,0.0
6,best,0,0,0.522879,0,0.60206,0.0,0.0,0.314804,0.0
7,mri,0,0,0,0.30103,0.60206,0.0,0.0,0.0,0.181238
8,retriev,0.522879,0.522879,0,0,0.30103,0.157402,0.157402,0.0,0.0
9,inform,0.522879,0,0,0.30103,0.30103,0.157402,0.0,0.0,0.090619


In [36]:
# loading data
import pandas as pd

tf_idf = pd.read_csv('unit-vector.csv')

tf_idf

,Terms,unit-vector 1,unit-vector 2,unit-vector 3,unit-vector 4,unit-vector 5,unit-vector 6,unit-vector 7,unit-vector 8,unit-vector 9,unit-vector 10
0,demand,0.000000,0.000000,0.0,-0.287915,0.0,0.000000,0.000000,0.000000,0.000000,0.0
1,prepar,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,-0.230354,0.000000,0.0
2,plan,0.000000,-0.476568,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
3,collect,-0.281615,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
4,human,0.000000,0.000000,0.0,0.000000,0.0,0.000000,-0.194431,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
125,degre,0.000000,0.000000,0.0,0.000000,0.0,-0.396227,0.000000,0.000000,0.000000,0.0
126,activ,-0.281615,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
127,techniqu,0.000000,-0.333107,0.0,0.000000,0.0,0.000000,-0.135902,0.000000,0.000000,0.0
128,practition,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,-0.239021,0.0


In [42]:
# tf-idf
import sys
import copy

rank = {}
def tfidf():
    for key,value in queries.items():
        result = {}
        for i in range(len(documents)):
            add=0
            for term in value:
                row_num = tf_idf.loc[tf_idf['Terms'] == term].index[0]
                temp='unit-vector '+str(i+1)
                add+=tf_idf.loc[row_num, temp]
            if key in rank:
                rank[key].append(add)
            else:
                rank[key] = [add]

        
    for key, value in rank.items():
        print('-----------------------')
        print(key)
        print('-----------------------')
        temp = copy.deepcopy(value)
        temp.sort(reverse=True)
        for i in temp:
            ind = value.index(i)
            value[ind] = -sys.maxsize
            print(f"Document {ind + 1}")

In [43]:
tfidf()

-----------------------
Query 1.txt
-----------------------
Document 2
Document 3
Document 6
Document 5
Document 7
Document 8
Document 9
Document 10
Document 4
Document 1
-----------------------
Query 2.txt
-----------------------
Document 2
Document 10
Document 1
Document 4
Document 5
Document 6
Document 8
Document 7
Document 9
Document 3
-----------------------
Query 3.txt
-----------------------
Document 6
Document 10
Document 5
Document 1
Document 2
Document 8
Document 4
Document 7
Document 3
Document 9
-----------------------
Query 4.txt
-----------------------
Document 3
Document 2
Document 6
Document 1
Document 5
Document 10
Document 4
Document 9
Document 8
Document 7


In [ ]:
# cosine similarity
import numpy as np

# define two vectors
vector1 = np.array([1, 2, 3])
vector2 = np.array([3, 4, 5])

# calculate the dot product of the two vectors
dot_product = np.dot(vector1, vector2)

# calculate the magnitude of vector1
mag1 = np.sqrt(np.sum(vector1**2))

# calculate the magnitude of vector2
mag2 = np.sqrt(np.sum(vector2**2))

# calculate the cosine similarity
cos_sim = dot_product / (mag1 * mag2)

# print the cosine similarity
print(cos_sim)

In [57]:
# cosine similarity
import numpy as np

rank = {}
j=0
for key,value in queries.items():
        vec2 = []
        q='tf-idf '+str(j+1)
        j+=1
        q_temp = list(matrix[q])
        for t in q_temp:
            if t!=0:
                vec2.append(t)
        for i in range(len(documents)):
            vec1 = []
            for term in value:
                row_num = tf_idf.loc[tf_idf['Terms'] == term].index[0]
                temp='unit-vector '+str(i+1)
                vec1.append(tf_idf.loc[row_num, temp])
            
            vector1 = np.array(vec1)
            vector2 = np.array(vec2)
            # calculate the dot product of the two vectors
            dot_product = np.dot(vector1, vector2)

            # calculate the magnitude of vector1
            mag1 = np.sqrt(np.sum(vector1**2))

            # calculate the magnitude of vector2
            mag2 = np.sqrt(np.sum(vector2**2))

            # calculate the cosine similarity
            cos_sim = dot_product / (mag1 * mag2)
            
            if math.isnan(cos_sim):
                    cos_sim = 0
            
            # print the cosine similarity
            if key in rank:
                rank[key].append(cos_sim)
            else:
                rank[key] = [cos_sim]
                

print(rank)
for key, value in rank.items():
        print('-----------------------')
        print(key)
        print('-----------------------')
        temp = copy.deepcopy(value)
        temp.sort(reverse=True)
        for i in temp:
            ind = value.index(i)
            value[ind] = -sys.maxsize
            print(f"Document {ind + 1}")

{'Query 1.txt': [-0.17445621815669637, 0.7611844082781005, 0.603261638710297, -0.816496580927726, 0, 0.8164965809277259, 0, 0, 0, 0], 'Query 2.txt': [0.3333333333333333, 0.3333333333333333, -0.3333333333333333, 0, 0, 0, -0.5303868328684584, -0.6666666666666666, -0.6666666666666666, 0.6666666666666666], 'Query 3.txt': [0, 0, -0.5773502691896257, -0.5773502691896257, 0.5773502691896258, 0.5773502691896258, -0.5773502691896257, 0, -0.7196610522187754, 0.5773502691896258], 'Query 4.txt': [0.48507125007266594, 0.48507125007266594, 0.485071250072666, -0.48507125007266594, 0, 0.485071250072666, -0.7001400420140048, -0.6157252668123478, -0.48507125007266594, 0]}
-----------------------
Query 1.txt
-----------------------
Document 6
Document 2
Document 3
Document 5
Document 7
Document 8
Document 9
Document 10
Document 1
Document 4
-----------------------
Query 2.txt
-----------------------
Document 10
Document 1
Document 2
Document 4
Document 5
Document 6
Document 3
Document 7
Document 8
Docume

C:\Users\fashn\AppData\Local\Temp\ipykernel_5836\611684826.py:33: RuntimeWarning: invalid value encountered in double_scalars
  cos_sim = dot_product / (mag1 * mag2)
